In [ ]:
## このプログラムは「Digit Recognizer」において作成した
## ノートブックで動作します

# データを用意する

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical

# train.csvを読み込んでpandasのDataFrameに格納
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train_x = train.drop(['label'], axis=1) # trainから画像データを抽出
train_y = train['label']                # trainから正解ラベルを抽出
test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

# trainのデータを学習データとテストデータに分ける。
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 画像のピクセル値を255.0で割って0～1.0の範囲にしてnumpy.arrayに変換
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

# 画像データの2階テンソルを
# (高さ = 28px, 幅 = 28px , チャンネル = 1)の
# 3階テンソルに変換
# グレースケールのためチャンネルは1。
tr_x = tr_x.reshape(-1,28,28,1)
va_x = va_x.reshape(-1,28,28,1)

# 正解ラベルをone-hot表現に変換
tr_y = to_categorical(tr_y, 10)
va_y = to_categorical(va_y, 10)

In [ ]:
# モデルを生成する
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D    # convolution layers

# Sequentialオブジェクトを生成
model = Sequential()

# 第1層のフィルター数、フィルターのサイズを探索
model.add(Conv2D(filters=32,
                 kernel_size=(5,5),
                 padding='same', 
                 activation='relu',
                 input_shape=(28,28,1)))

# 第2層のフィルター数、フィルターのサイズを探索
model.add(Conv2D(filters = 64,
                 kernel_size = (7,7),
                 padding='same', 
                 activation='relu'))

# 第3層に2×2のプーリング層を配置。
model.add(MaxPooling2D(pool_size=(2,2)))

# ドロップアウト率を探索。
model.add(Dropout(0.5))

# 第4層のフィルター数、フィルターのサイズを探索
model.add(Conv2D(filters=64,
                 kernel_size=(5,5),
                 padding='same', 
                 activation='relu'))

# 第5層のフィルター数、フィルターのサイズを探索
model.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding='same', 
                 activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# ドロップアウト率を探索。
model.add(Dropout(0.55))

# Flatten層を配置。
model.add(Flatten())

# 第6層
model.add(Dense(700, activation='relu'))
model.add(Dropout(0.3))

# 第7層
model.add(Dense(150, activation='relu'))
model.add(Dropout(0.35))

# 出力層を配置。
model.add(Dense(10, activation = "softmax"))

# モデルのコンパイル。
# オプティマイザーはAdam
model.compile(loss="categorical_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])

# モデルの構造を出力。
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# データ拡張
datagen = ImageDataGenerator(
    width_shift_range=0.1,  # 横サイズの0.1の割合でランダムに水平移動
    height_shift_range=0.1, # 縦サイズの0.1の割合でランダムに垂直移動
    rotation_range=10,      # 10度の範囲でランダムに回転させる
    zoom_range=0.1          # ランダムに拡大
    )
 
# ミニバッチのサイズ
batch_size = 100
# 学習回数
epochs = 20

# 学習を行う
history = model.fit(
    # 拡張データをミニバッチの数だけ生成
    # 出力は正規化される
    datagen.flow(tr_x,
                 tr_y,
                 batch_size=batch_size),
    # 1回の学習におけるステップ数
    # 画像の枚数をミニバッチのサイズで割った整数値
    steps_per_epoch=tr_x.shape[0] // batch_size,
    epochs=epochs, # 学習回数
    verbose=1,     # 学習の進捗状況を出力する
    # テストデータ
    validation_data=(va_x,va_y))